In [ ]:
%%capture
!pip install git+https://github.com/vyperlang/titanoboa.git@master

In [ ]:
from google.colab import userdata
import boa

## **Setting Up the Infrastructure**

Before we can simulate some transactions, we need to initialize the necessary contract. For API keys, we can use the "Secrets" feature of Google Colab. An `RPC_ETHEREUM` including an HTTP key (e.g., from Alchemy) and an `ETHERSCAN_API_KEY` including an Etherscan API key need to be created before the notebook can be used.

Interacting with the contract is very simple when using titanoboa. The `from_etherscan` function lets us connect to the contracts.

`boa.env.fork` forks the desired chain for us.

Official Titanoboa Documentation: [https://titanoboa.readthedocs.io/en/latest/](https://titanoboa.readthedocs.io/en/latest/)


In [ ]:
boa.env.fork(userdata.get('RPC_ETHEREUM'))

VAULT = '0xcea18a8752bb7e7817f9ae7565328fe415c0f2ca'
CRVUSD = '0xf939E0A03FB07F59A73314E73794Be0E57ac1b4E'

vault = boa.from_etherscan(
    VAULT,
    name='crvUSD Vault for CRV Lending Market',
    api_key=userdata.get('ETHERSCAN_API_KEY')
)

crvusd = boa.from_etherscan(
    CRVUSD,
    name='crvUSD',
    api_key=userdata.get('ETHERSCAN_API_KEY')
)

vault, crvusd

---

## **Withdrawing and Redeeming**

There are two functions that let users withdraw the underlying assets from the vault:

- `withdraw`: Lets a user withdraw a desired amount of assets from the vault by burning shares.
- `redeem`: Lets a user redeem (and burn) a specific amount of shares and in exchange receive the corresponding amount of assets.

---

_Before being able to withdraw assets and redeem shares, the user needs to grant approval to the contract to spend the shares:_

NOTE: In this case, we need to grant approval to the vault to spend shares, and we do not grant maximum approval. Instead, we only grant approval for the token balance of the USER.


In [ ]:
USER = '0xa3CcD5671E893D9c8d5c32f3B65D2cE5fceFc6D6' #random user with vault shares and ETH for gas

share_balance = vault.balanceOf(USER)

with boa.env.prank(USER):
  vault.approve(vault, share_balance)
  assert vault.allowance(USER, VAULT) == share_balance

print("Amount of Shares approved for the Vault to spend:", vault.allowance(USER, VAULT))

---

### **`withdraw`**

`Vault.withdraw(assets: uint256, receiver: address = msg.sender, owner: address = msg.sender) -> uint256:`

Function to withdraw `assets` from `owner` to the `receiver` and burn the corresponding amount of shares.

Returns: shares withdrawn (`uint256`).

| Input      | Type      | Description                                               |
|------------|-----------|-----------------------------------------------------------|
| `assets`   | `uint256` | Amount of assets to withdraw.                             |
| `receiver` | `address` | Receiver of the assets. Defaults to `msg.sender`.         |
| `owner`    | `address` | Address of whose shares to burn. Defaults to `msg.sender`.|

Withdrawing assets (or redeeming shares) from the vault is generally always possible. However, there might be cases where a lending market has very high utilization, which could hinder the withdrawal of assets. For example, if a market has a utilization rate of 100%, meaning every supplied asset in the vault is borrowed, then a user cannot redeem their vault shares for assets. They would need to wait for the utilization rate to go down.

To prevent this scenario, the borrow rate is based on the utilization rate of the lending market. If the utilization reaches 100%, this would cause the interest rate to skyrocket to the maximum value, incentivizing either the borrowers to repay debt or lenders to supply more assets to the vault.

---

Before a user withdraws assets, they might want to know how much a single share is worth. For that, the vault provides an easy `convertToAssets` function.

Additionally, there is a `previewWithdraw` function which essentially simulates a withdrawal action and returns the amount of shares burned in the process.


In [ ]:
# to check how many much 10,000 share is worth:
print("10,000 vault shares are worth approximately", round(vault.convertToAssets(10000 * 10**18) / 1e18, 6), "crvUSD.")

# simulate the withdrawl of 10 crvUSD
print("Shares burned when withdrawing 10 crvUSD:", round(vault.previewWithdraw(10 * 10**18) / 1e18, 2))

In [ ]:
def withdraw_assets(assets, receiver, owner):

  crvusd_balance_before = crvusd.balanceOf(USER)
  share_balance_before = vault.balanceOf(USER)
  print("crvusd balance before deposit", round(crvusd_balance_before / 1e18, 2))
  print("share balance before deopsit", round(share_balance_before / 1e18, 2))

  with boa.env.prank(USER):
    # withdraw assets from the vault
    vault.withdraw(assets, receiver, owner)

  crvusd_balance_after = crvusd.balanceOf(USER)
  share_balance_after = vault.balanceOf(USER)
  print("crvusd balance after deposit", round(crvusd_balance_after / 1e18, 2))
  print("share balance after deopsit", round(share_balance_after / 1e18, 2))

  print("--------------------------")
  print("crvusd difference:", round((crvusd_balance_after - crvusd_balance_before) / 1e18, 2))
  print("share difference:", round((share_balance_after - share_balance_before) / 1e18, 2))

assets_to_withdraw = 1000 * 10**18
receiver = USER
owner = USER

withdraw_assets(assets_to_withdraw, receiver, owner)

---

### **`redeem`**

`Vault.redeem(shares: uint256, receiver: address = msg.sender, owner: address = msg.sender) -> uint256:`

Function to redeem (and burn) `shares` from `owner` and send the received assets to `receiver`. Shares are burned when they are redeemed.

Returns: assets received (`uint256`).

| Input      | Type      | Description                                               |
|------------|-----------|-----------------------------------------------------------|
| `shares`   | `uint256` | Amount of shares to redeem.                               |
| `receiver` | `address` | Receiver of the assets. Defaults to `msg.sender`.         |
| `owner`    | `address` | Address of whose shares to burn. Defaults to `msg.sender`.|

In [ ]:
def redeem_shares(assets, receiver, owner):

  crvusd_balance_before = crvusd.balanceOf(USER)
  share_balance_before = vault.balanceOf(USER)
  print("crvusd balance before deposit", round(crvusd_balance_before / 1e18, 2))
  print("share balance before deopsit", round(share_balance_before / 1e18, 2))

  with boa.env.prank(USER):
    # redeem shares
    vault.redeem(assets, receiver, owner)

  crvusd_balance_after = crvusd.balanceOf(USER)
  share_balance_after = vault.balanceOf(USER)
  print("crvusd balance after deposit", round(crvusd_balance_after / 1e18, 2))
  print("share balance after deopsit", round(share_balance_after / 1e18, 2))

  print("--------------------------")
  print("crvusd difference:", round((crvusd_balance_after - crvusd_balance_before) / 1e18, 2))
  print("share difference:", round((share_balance_after - share_balance_before) / 1e18, 2))


# for redeeming all of the users shares, we can just redeem balanceOf(USER)
shares_to_redeem = vault.balanceOf(USER)
receiver = USER
owner = USER

redeem_shares(shares_to_redeem, receiver, owner)